<a href="https://colab.research.google.com/github/Bene939/BERT_News_Sentiment_Classifier/blob/main/BERT_News_Sentiment_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch
!pip install pandas
!pip install pathlib
!pip install sklearn
!pip install numpy

     |████████████████████████████████| 1.3MB 13.6MB/s 
     |████████████████████████████████| 1.1MB 50.9MB/s 
     |████████████████████████████████| 890kB 62.0MB/s 
     |████████████████████████████████| 2.9MB 52.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=dbb0285078e26135efe08e8b72dcb086909b46714f9fe41828cfac77c00a7402
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import BertModel, DistilBertModel, BertForSequenceClassification, AdamW, BertTokenizer, get_linear_schedule_with_warmup, Trainer, TrainingArguments
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import pandas as pd
from pathlib import Path
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from torch.nn import functional as F
from collections import defaultdict
import random

In [ ]:
# setting device as gpu or cpu

if torch.cuda.is_available():
  print("\nUsing: ", torch.cuda.get_device_name(0))
  device = torch.device('cuda')
else:
  print("\nUsing: CPU")
  device = torch.device('cpu')

In [ ]:
#TODO: compare balanced vs unbalanced dataset performance (might not be necessary: pretty balanced already)
#TODO: balancing dataset with pandas

# loading dataset
labeled_dataset = "news_headlines_sentiment.csv"
labeled_dataset_file = Path(labeled_dataset)
file_loaded = False
while not file_loaded:
  if labeled_dataset_file.exists():
    labeled_dataset = pd.read_csv(labeled_dataset_file)
    file_loaded = True
    print("Dataset Loaded")
  else:
    print("File not Found")
print(labeled_dataset)
print(labeled_dataset.sentiment.value_counts())
"""

# balancing dataset to 1/3 per sentiment
for idx, row in labeled_dataset.iterrows():
  if row["sentiment"] == 0:
    if negative - neutral != 0:
      index_name = labeled_dataset[labeled_dataset["news"] == row["news"]].index
      labeled_dataset.drop(index_name, inplace=True)
      negative -= 1
  elif row["sentiment"] == 2:
    if positive - neutral != 0:
      index_name = labeled_dataset[labeled_dataset["news"] == row["news"]].index
      labeled_dataset.drop(index_name, inplace=True)
      positive -= 1
"""

In [ ]:
# loading phrase bank dataset and correcting format

phrase_bank_dataset = "all-data.csv"
phrase_bank_dataset_file = Path(phrase_bank_dataset)
file_loaded = False
while not file_loaded:
  if phrase_bank_dataset_file.exists():
    phrase_dataset = pd.read_csv(phrase_bank_dataset, encoding='latin-1', names=["sentiment", "news"])
    phrase_dataset = phrase_dataset[["news", "sentiment"]]
    phrase_dataset["sentiment"].replace(['positive', 'negative', 'neutral'], [0,1,2], inplace=True)
    file_loaded = True
    print("Dataset Loaded")
  else:
    print("File not Found")
print(phrase_dataset)
print(phrase_dataset.sentiment.value_counts())

In [ ]:
# merge both datasets
merged_dataset = pd.concat([phrase_dataset, labeled_dataset], axis=0)
print(merged_dataset)
print(merged_dataset.sentiment.value_counts())

In [ ]:
# custom dataset class which returns the encodings and labels when called by the data loader
# code is from https://huggingface.co/transformers/custom_datasets.html

class NewsSentimentDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

  def __getitem__(self, idx):
      item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
      item['labels'] = torch.tensor(self.labels[idx])
      return item

  def __len__(self):
      return len(self.labels)

In [ ]:
# expects data frame and tokenizer as input
# returns encoded data as NewsSentimentDataset
# token type ids are not included in encoding as they are only for Q&A to mark if its a question or an answer

def tokenize_headlines(df, tokenizer):
  encodings = tokenizer.batch_encode_plus(
      df["news"].tolist(),           # input the news headlines
      add_special_tokens = True,     # special tokens added to mark beginning & end of sentence
      truncation = True,             # make all sentences a fixed length
      padding = 'max_length',        # pad with zeros to max length
      return_attention_mask = True,  # include attention mask in encoding
      return_tensors = 'pt'          # return as pytorch tensor
  )

  dataset = NewsSentimentDataset(encodings, df["sentiment"].tolist())
  return dataset

In [ ]:
#splitting dataset into training and validation set
#load news sentiment dataset
#defining tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
train_data, val_data = train_test_split(merged_dataset, test_size=.2)

print("Train Dataset\n", train_data.reset_index(drop=True))
print("Validation Dataset\n", val_data.reset_index(drop=True))

train_dataset = tokenize_headlines(train_data, tokenizer)
val_dataset = tokenize_headlines(val_data, tokenizer)

In [ ]:
#TODO: Calculate per class accuracy
#TODO: Try out different Loss function
#TODO: Optimizer zero grad etc necessary?
#TODO: Cased vs Uncased

model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=3)
model = model.to(device)
#data loader
train_batch_size = 8
val_batch_size = 8

train_data_loader = DataLoader(train_dataset, batch_size = train_batch_size, sampler=RandomSampler(train_dataset))
val_data_loader = DataLoader(val_dataset, batch_size = val_batch_size, sampler=SequentialSampler(val_dataset))

#optimizer and scheduler
num_epochs = 1
num_steps = len(train_data_loader) * num_epochs
optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=1000, num_training_steps=num_steps)

#training and evaluation
seed_val = 64
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

for epoch in range(num_epochs):

  print("\n###################################################")
  print("Epoch: {}/{}".format(epoch+1, num_epochs))
  print("###################################################\n")

  #training phase
  average_train_loss = 0
  average_train_acc = 0
  for step, batch in enumerate(train_data_loader):

    model.train() 
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    # set the gradient to zero as by default pytorch is accumulating the gradients
    model.zero_grad()
    optimizer.zero_grad()

    loss, logits = model(input_ids=input_ids,
                   attention_mask=attention_mask,
                   labels=labels)

    #loss = F.cross_entropy(logits, labels)
    average_train_loss += loss
    
    if step % 100 == 0:
      print("At Step {} Training Loss: {:.5f}".format(step, loss.item()))

    
    loss.backward()
    #maximum gradient clipping
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    
    optimizer.step()
    scheduler.step()
    model.zero_grad()

    logits_for_acc = logits.detach().cpu().numpy()
    label_for_acc = labels.to('cpu').numpy()
    average_train_acc += sklearn.metrics.accuracy_score(label_for_acc, np.argmax(logits_for_acc, axis=-1))

    #print out sentences + sentiment predictions + labels
    #print(tokenizer.batch_decode(input_ids, skip_special_tokens=True))
    #print("Predictions: ",np.argmax(logits_for_acc, axis=1))
    #print("Labels:      ",label_for_acc)
    #print("#############")
      

  average_train_loss = average_train_loss / len(train_data_loader)
  average_train_acc = average_train_acc / len(train_data_loader)
  print("======Average Training Loss: {:.5f}=========".format(average_train_loss))
  print("======Average Training Accuracy: {:.2f}%========".format(average_train_acc*100))

  #validation phase
  average_val_loss = 0
  average_val_acc = 0
  
  for step,batch in enumerate(val_data_loader):
    model.eval()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    with torch.no_grad():
      loss, logits = model(input_ids=input_ids,
                     attention_mask=attention_mask,
                     labels=labels)

    #loss = F.cross_entropy(logits, labels)
    average_val_loss += loss.item()

    logits_for_acc = logits.detach().cpu().numpy()
    label_for_acc = labels.to('cpu').numpy()
    #print out sentences + sentiment predictions + labels
    #print(tokenizer.batch_decode(input_ids, skip_special_tokens=True))
    #print("Predictions: ",np.argmax(logits_for_acc, axis=1))
    #print("Labels:      ",label_for_acc)
    #print("#############")
    average_val_acc += sklearn.metrics.accuracy_score(label_for_acc, np.argmax(logits_for_acc, axis=-1))
    

  average_val_loss = average_val_loss / len(val_data_loader)
  average_val_acc = average_val_acc / len(val_data_loader)
  print("======Average Validation Loss: {:.5f}=========".format(average_val_loss))
  print("======Average Validation Accuracy: {:.2f}%======".format(average_val_acc*100))
